In [1]:
from experta import *
import numpy as np

# each Move object contains the white and black move
class Move(Fact):
    number = Field(int, mandatory=True)  # mandatory means required
    white = Field(str, mandatory=True)
    black = Field(str, default='')
    is_variant = Field(int, default=0) # 0 no variant, 1 for white, 2 for black
    variant = Field(list, default=[]) # list of branches index when there's variant

class Branch(Fact):
    name = Field(str, mandatory=True)
    moves = Field(list, default=[])

class Opening(Fact):
    name = Field(str, mandatory=True)
    branches = Field(list, default=[])
    color = Field(str, default='both')

class Path(Fact):
    """My playing path"""
    moves = Field(list, default=[])

class Questionnaire(Fact):
    """Questions to determine playing style."""
    question = Field(str, mandatory=True)

class CertaintyFactor(Fact):
    """Chess related facts with certainty factors."""
    name = Field(str, mandatory=True)
    cf = Field(float, default=100.0)


# Define the engine
class ChessOpeningEngine(KnowledgeEngine):

    def __init__(self):
        super().__init__()
        self.move_number = 0

    @DefFacts()
    def _initial_action(self):
        yield Fact(end=False)
        yield Path(moves=[])
        yield Opening(name='Ruy Lopez', color='white', branches=[
            Branch(name='Main Line', moves=[
                Move(number=0, white='e4', black='e5', is_variant=0),
                Move(number=1, white='Nf3', black='Nc6', is_variant=0),
                Move(number=2, white='Bb5', black='a6', is_variant=2, variant=[1, 2]),
                Move(number=3, white='Ba4', black='d6', is_variant=0),
                Move(number=4, white='O-O', black='b5', is_variant=0),
                Move(number=3, white='Bb3', black='Bg4', is_variant=0),
                Move(number=3, white='h3', black='Bf3', is_variant=0),
            ]),
            Branch(name='The Old Steinitz Defense', moves=[
                Move(number=0, white='e4', black='e5', is_variant=0),
                Move(number=1, white='Nf3', black='Nc6', is_variant=0),
                Move(number=2, white='Bb5', black='d6', is_variant=0),
                Move(number=3, white='d4', black='exd4', is_variant=0),     
            ]),
            Branch(name='The Bird\'s Defense', moves=[
                Move(number=0, white='e4', black='e5', is_variant=0),
                Move(number=1, white='Nf3', black='Nc6', is_variant=0),
                Move(number=2, white='Bb5', black='Nd4', is_variant=0),
                Move(number=3, white='NxNd4', black='d4', is_variant=0),     
            ]),
        ])

        yield Opening(name='Caro-Kann Defense', color='black', branches=[
            Branch(name='Main Line', moves=[
                Move(number=1, white='e4', black='e5'),
                Move(number=2, white='Nf3', black='Nc6'),
                Move(number=3, white='Bb5', black='a6'),
            ]),
        ])
        yield Opening(name='Queen\'s Gambit', color='white', branches=[])

    # Start
    @Rule(salience=100)
    def start(self):
        color = input("Do you want to play as White or Black? ").strip().lower()
        self.declare(Fact(color=color))

    # List Openings
    @Rule(Fact(color=MATCH.mycolor))
    def list_openings(self, mycolor):
        print(f"You chosed to play as {mycolor}.\n")
        print(f"{mycolor} Openings:")
        for opening in self.facts.values():
            if isinstance(opening, Opening) and opening['color'] in (mycolor, 'both'):
                print(opening['name'], flush=True)
        choosen_opening = input('Choose an opening: ')
        self.declare(Fact(choosen_opening=choosen_opening))
        self.declare(Fact(current_branch=0))
 
    # For testing
    @Rule(Fact(choosen_opening=MATCH.name),
          AS.opening << Opening(name=MATCH.name),
          salience=10
        )
    def print_opening(self, opening):
        branches = opening['branches']
        for branch in branches:
            print(f'\n{opening["name"]} - {branch["name"]}:')
            for move in branch["moves"]:
                print(f"{move['number']}. {move['white']} {move['black']}")
        print()

    # Play
    @Rule(Fact(choosen_opening=MATCH.name),
           Opening(name=MATCH.name, branches=MATCH.branches),
           AS.current_branch << Fact(current_branch=MATCH.branch_number),
           AS.path << Path(moves=MATCH.moves),
           AS.endFact << Fact(end=L(False)),
           salience=5
        )
    def play(self, branches, path, endFact, current_branch, branch_number):
        branch = branches[branch_number]  # First time: Main Line
        new_moves = list(path['moves'])  # Use path's moves
        current_move = branch['moves'][self.move_number]  # Access the Move object

        # Choose for white
        white_move, white_branch = self.choose_white(current_move, branches)
        
        # Choose for black
        black_move, black_branch = self.choose_black(current_move, branches)

        chosen_move = Move(number=self.move_number, white=white_move, black=black_move)
        
        new_moves.append(chosen_move)
        self.modify(path, moves=new_moves)
        self.move_number += 1

        # Update current branch if a variant was chosen
        if white_branch is not None:
            self.modify(current_branch, current_branch=white_branch)
        elif black_branch is not None:
            self.modify(current_branch, current_branch=black_branch)

        if self.move_number >= len(branch['moves']):
            self.modify(endFact, end=True)

    # Choose move for white
    def choose_white(self, current_move, branches):
        possible_moves = [(current_move["white"], None)]
        if current_move.get('is_variant') == 1:  # Use .get() to avoid KeyError
            for v in current_move['variant']:
                branch = branches[v]
                possible_moves.append((branch['moves'][self.move_number]['white'], v))

        print(f'Possible white moves: {possible_moves}', flush=True) # This forces the print buffer to flush immediately, ensuring that all printed messages appear before the input prompt.
        chosen_move = input('Choose white move: ').strip()
        for move, branch in possible_moves:
            if move == chosen_move:
                return move, branch
        return chosen_move, None
    
    # Choose move for black
    def choose_black(self, current_move, branches):
        possible_moves = [(current_move["black"], None)]
        if current_move.get('is_variant') == 2:  # Use .get() to avoid KeyError
            for v in current_move['variant']:
                branch = branches[v]
                possible_moves.append((branch['moves'][self.move_number]['black'], v))

        print(f'Possible black moves: {possible_moves}', flush=True)
        chosen_move = input('Choose black move: ').strip()
        for move, branch in possible_moves:
            if move == chosen_move:
                return move, branch
        return chosen_move, None
        

engine = ChessOpeningEngine()
engine.reset()
engine.run()
engine.facts

You chosed to play as black.

black Openings:
Caro-Kann Defense

Caro-Kann Defense - Main Line:
1. e4 e5
2. Nf3 Nc6
3. Bb5 a6

Possible white moves: [('e4', None)]
Possible black moves: [('e5', None)]
Possible white moves: [('Nf3', None)]
Possible black moves: [('Nc6', None)]
Possible white moves: [('Bb5', None)]
Possible black moves: [('a6', None)]


FactList([(0, InitialFact()),
          (3,
           Opening(name='Ruy Lopez', color='white', branches=frozenlist([<frozendict {'name': 'Main Line', 'moves': frozenlist([<frozendict {'number': 0, 'white': 'e4', 'black': 'e5', 'is_variant': 0}>, <frozendict {'number': 1, 'white': 'Nf3', 'black': 'Nc6', 'is_variant': 0}>, <frozendict {'number': 2, 'white': 'Bb5', 'black': 'a6', 'is_variant': 2, 'variant': frozenlist([1, 2])}>, <frozendict {'number': 3, 'white': 'Ba4', 'black': 'd6', 'is_variant': 0}>, <frozendict {'number': 4, 'white': 'O-O', 'black': 'b5', 'is_variant': 0}>, <frozendict {'number': 3, 'white': 'Bb3', 'black': 'Bg4', 'is_variant': 0}>, <frozendict {'number': 3, 'white': 'h3', 'black': 'Bf3', 'is_variant': 0}>])}>, <frozendict {'name': 'The Old Steinitz Defense', 'moves': frozenlist([<frozendict {'number': 0, 'white': 'e4', 'black': 'e5', 'is_variant': 0}>, <frozendict {'number': 1, 'white': 'Nf3', 'black': 'Nc6', 'is_variant': 0}>, <frozendict {'number': 2, 'white': 'Bb

**Chess Openings**
